# 拆书系统 - Kaggle Worker 节点

## 使用步骤

### 1. Kaggle 设置
- **Accelerator**: CPU (不要选 GPU)
- **Internet**: **ON** (必需)

### 2. 上传 .env 文件
- 点击右侧 **+ Add Data** → **Upload**
- 上传你的 `.env` 配置文件

### 3. 运行
依次运行下面的 4 个单元格即可。

---

## 1. 安装依赖包

In [ ]:
%%time
!pip install -q Flask==2.3.3 Flask-CORS==4.0.0 Flask-SocketIO==5.3.6 SQLAlchemy==2.0.21 \
    httpx==0.25.0 PyMySQL==1.1.0 neo4j==5.28.0 redis==5.0.0 python-dotenv==1.0.0 \
    APScheduler==3.11.0 requests==2.32.0 chardet==5.2.0

print("✅ 依赖包安装完成")

## 2. 下载代码并配置

In [ ]:
%%time
import os
import shutil

# Clone 代码
print("📦 下载代码...")
!git clone https://github.com/ronghuaxueleng/chaishu-worker-service.git /kaggle/working/worker

# 切换目录
os.chdir('/kaggle/working/worker')
print(f"✅ 工作目录: {os.getcwd()}")

# 复制 .env 文件
if os.path.exists('/kaggle/input/chaisuh-env/.env'):
    shutil.copy('/kaggle/input/chaisuh-env/.env', '.env')
    print("✅ 配置文件已复制")
else:
    raise FileNotFoundError("❌ 请先上传 .env 文件")

# 创建日志目录
!mkdir -p logs
print("✅ 环境准备完成")

## 3. 查看配置（可选）

In [ ]:
# 显示配置（隐藏密码）
print("📋 配置信息:")
print("=" * 50)
with open('.env') as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, val = line.split('=', 1)
            if 'PASSWORD' in key.upper():
                print(f"{key}=***")
            else:
                print(f"{key}={val}")
print("=" * 50)

## 4. 启动 Worker

**⚠️ 此单元格会持续运行直到手动停止或 Kaggle 超时（9小时）**

- 点击左侧 **停止按钮** 可以停止
- Worker 会自动发现所有激活的 AI providers 并启动对应进程
- 进程数由 `.env` 中的 `KG_WORKERS_PER_PROVIDER` 控制

In [ ]:
# 启动 Worker（会一直运行）
# 设置 spawn 模式以兼容 Kaggle/Jupyter 环境
import os
os.environ['MULTIPROCESSING_START_METHOD'] = 'spawn'

!python worker.py

---

## 故障排查

如果遇到问题，运行下面的单元格查看日志。

In [ ]:
# 查看最新 100 行日志
!tail -n 100 logs/worker.log

## 停止 Worker

In [ ]:
# 停止所有 worker 进程
!pkill -f "python worker.py"
print("✅ Worker 已停止")